In [81]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Initial steps if running on Google Colab, to download support files from GitHub and set the working directory

In [82]:
#for running on Colab
#!git clone https://github.com/MBWestcott/gas-forecast.git

# 2. Change into the repo directory
%cd /content/gas-forecast/notebooks



[WinError 3] The system cannot find the path specified: '/content/gas-forecast/notebooks'
d:\dev\gas-forecast\notebooks


### First download the raw data from the National Gas data portal

In [83]:

raw_data_folder = Path("../data/raw/")

def download_csv(url, output_file):
    """
    Downloads a CSV file from the given URL and saves it to the specified file.

    :param url: URL to download the CSV data from.
    :param output_file: Path to the local file where the CSV will be saved.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Ensure we notice bad responses

        # Write the content (CSV data) to a file in binary mode
        with open(output_file, 'wb') as f:
            f.write(response.content)

        print(f"CSV file has been successfully downloaded and saved as '{output_file}'.")

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")


def download_raw_data():
    pubIdsFile = Path("../PUB ids.txt")
    with open(pubIdsFile) as f:
        pubIds = f.read()
        pubIds = pubIds.replace("\n", ",").strip()

    earliest = datetime.date(2020,4,1) # Download data going back 5 years
    
    download_from = datetime.date.today().replace(day=1) # start first download on first day of current month
    download_to = datetime.date.today() # end first download on today's date
    while(download_from > earliest):

        # Format the date in yyyy-mm-dd format
        formatted_from = download_from.strftime("%Y-%m-%d")
        formatted_to = download_to.strftime("%Y-%m-%d")

        csv_url = f"https://data.nationalgas.com/api/find-gas-data-download?applicableFor=Y&dateFrom={formatted_from}&dateTo={formatted_to}&dateType=GASDAY&latestFlag=Y&ids={pubIds}&type=CSV"
        month_format = download_from.strftime("%Y-%m")
        output_filename = raw_data_folder /  f"{month_format}.csv"

        download_csv(csv_url, output_filename)
        time.sleep(2) # brief courtesy sleep
        download_to = download_from - datetime.timedelta(days=1) # next download should go up to the day before the previous download start date
        download_from = download_to.replace(day=1) # next download should start on the first day of the month

# Do the download if the raw data is not there already
csvCount = sum(1 for f in raw_data_folder.iterdir() if f.is_file() and f.suffix == '.csv')
if(csvCount < 60):
    download_raw_data()

### Load the raw data

Load the raw CSVs into a single and dataframe, pivot it so that each column represents a feature.
Rename the Applicable At date field to Gas Day, and rename the columns that are going to be reused for ground truth and time series

In [84]:
label_cols = ["SAP", "SMPBuy", "SMPSell"]

def pivot(df : pd.DataFrame, cols):

    #only keep the values we are interested in
    mask = df["Data Item"].isin(cols)

    df_filtered = df[mask]

    # if there are duplicates for the field and gas day, take the latest
    df_latest = (
        df_filtered
        .sort_values("Applicable At")
        .groupby(["Gas Day", "Data Item"])
        .last()  # this takes the row with the highest (i.e. latest) "Applicable At" per group
        .reset_index()
    )

    # pivot to get 1 row per gas day
    df_latest = df_latest.pivot(index="Gas Day", columns="Data Item", values="Value").reset_index()

    # Drop 1 column that accounts for most of the NaNs
    #df_latest.drop(columns=["Composite Weather Variable - Actual"], inplace=True)

    return df_latest

def load_data():
    #Read raw CSVs
    pathlist = list(Path(raw_data_folder).rglob('*.csv'))
    file_count = len(pathlist)
    dfs = []
    files_done = 0
    for path_obj in pathlist:
        path = str(path_obj)

        df = pd.read_csv(path,
            parse_dates=["Applicable At", "Applicable For", "Generated Time"],
            dayfirst=True)

        df.rename(columns={'Applicable For': 'Gas Day'}, inplace=True)
        df['Gas Day'] = pd.to_datetime(df['Gas Day'], dayfirst=True)
        # pivot to 1 row per gas day, with features as columns

        daily_cols = df["Data Item"].unique()

        df_daily = pivot(df, daily_cols)
        dfs.append(df_daily)

        files_done += 1
        if files_done % 10 == 0:
            print(f"Processed {files_done} of {file_count} raw files")

    df = pd.concat(dfs)

    #Rename the columns that are going to be reused for ground truth and time series
    df.rename(columns={"SAP, Actual Day": 'SAP', "SMP Buy, Actual Day": 'SMPBuy', "SMP Sell, Actual Day": 'SMPSell'}, inplace=True)
    return df

df = load_data()
df.to_csv(Path("../data/processed/pivoted.csv"), index=False)
df.info()

Processed 10 of 60 raw files
Processed 20 of 60 raw files
Processed 30 of 60 raw files
Processed 40 of 60 raw files
Processed 50 of 60 raw files
Processed 60 of 60 raw files
<class 'pandas.core.frame.DataFrame'>
Index: 1819 entries, 0 to 22
Data columns (total 45 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   Gas Day                                                   1819 non-null   datetime64[ns]
 1   Aggregate LNG Importations - Daily Flow                   1816 non-null   float64       
 2   Beach Including Norway - Daily Flow                       1816 non-null   float64       
 3   Beach and IOG - Beach Delivery                            1815 non-null   float64       
 4   Beach and IOG - Daily Flow                                1816 non-null   float64       
 5   Composite Weather Variable - Actual                       1554 

### Preprocess data

Add the previous 5 days' prices as lag features, and 7- and 30-day rolling averages and standard deviations. Also add day of week features, and a cyclical coding of the day of year for seasonality.

In [85]:
def preprocess(df: pd.DataFrame, add_lags=True, add_labels=True):

    """Deal with missing values, add lagged features, rolling averages and stds, Day of Week, and cyclic encoding for seasonality"""

    if add_lags:
        lag_days = 5
        for i in range(1, lag_days+1):
            for col in label_cols:
                df[f"{col} D-{i}"] = df[col].shift(i)

        # add rolling averages and stds
        for col in label_cols:
            for window in [7, 30]:
                df[f'{col} D{window} roll mean'] = (
                    df[col]
                    .shift(1)               # so today's feature doesn't include today's price
                    .rolling(window=window, min_periods=1)  
                    .mean()
                    )
                df[f'{col} D{window} roll std'] = (
                    df[col]
                    .shift(1)               # so today's feature doesn't include today's price
                    .rolling(window=window, min_periods=1)  
                    .std()
                )

    # add day of week
    df['Day of Week'] = df['Gas Day'].dt.weekday
    df['Is Weekday'] = (df['Gas Day'].dt.weekday < 5).astype(int)
    df['Next Day Is Weekday'] = ((df['Gas Day'] + pd.Timedelta(days=1)).dt.weekday < 5).astype(int)
    # cyclic encoding for seasonality
    df['Day of Year'] = df['Gas Day'].dt.dayofyear
    df['sin_DoY'] = np.sin(2 * np.pi * df['Day of Year'] / 365)
    df['cos_DoY'] = np.cos(2 * np.pi * df['Day of Year'] / 365)

    if add_labels:
        # Add labels for next day's actuals
        for col in label_cols:
            df[f"Next Day {col}"] = df[col].shift(-1)

    return df

df = preprocess(df)
df.to_csv(Path("../data/processed/preprocessed.csv"), index=False)
df.head()

Data Item,Gas Day,Aggregate LNG Importations - Daily Flow,Beach Including Norway - Daily Flow,Beach and IOG - Beach Delivery,Beach and IOG - Daily Flow,Composite Weather Variable - Actual,Composite Weather Variable - Cold,Composite Weather Variable - Normal,Composite Weather Variable - Warm,Demand - Cold,...,SMPSell D30 roll std,Day of Week,Is Weekday,Next Day Is Weekday,Day of Year,sin_DoY,cos_DoY,Next Day SAP,Next Day SMPBuy,Next Day SMPSell
0,2020-05-01,66.15110,135.26344,201.41454,201.41454,10.5824,7.85,11.36,14.94,268.090483,...,NaN,4,1,0,122,0.863142,-0.504961,0.4770,0.5123,0.4417
1,2020-05-02,58.78630,131.66283,190.44913,190.44913,11.3089,7.99,11.47,15.02,244.938804,...,NaN,5,0,0,123,0.854322,-0.519744,0.4840,0.5193,0.4487
2,2020-05-03,56.55015,141.57363,198.12378,198.12378,11.6531,8.13,11.55,15.09,242.854588,...,0.003748,6,0,1,124,0.845249,-0.534373,0.4720,0.5073,0.4367
3,2020-05-04,52.82721,152.87412,205.70133,205.70133,11.9252,8.26,11.65,15.14,242.098717,...,0.006170,0,1,1,125,0.835925,-0.548843,0.4790,0.5143,0.4437
4,2020-05-05,62.21188,134.50813,196.72001,196.72001,11.4803,8.40,11.76,15.18,247.112422,...,0.005755,1,1,1,126,0.826354,-0.563151,0.5017,0.5370,0.4664


### Clean missing values and outliers

Most of the missing values are missing "Composite Weather Variable - Actual" from 2020-21. These affect around 15% of the dataset. Best way to fill in those is with the Normal forecast, which should usually be the closest. Apart from that there are very few missing readings so it is feasible to discard any remaining rows with missing data (done at the end, to avoid introducing errors into the lag features)

Also remove outliers where any of the prices was 0, and one of the next day prices was more 50% away from the current day's price

In [86]:
def clean(df: pd.DataFrame, remove_outliers=True):
    # fill missing CWV actuals with the normal forecast
    df['Composite Weather Variable - Actual'] = df['Composite Weather Variable - Actual'].fillna(df['Composite Weather Variable - Normal'])

    # There should be very remaining few rows that have any NaNs so we can drop any that do
    df.dropna(inplace=True)

    # Can drop the composite weather forecasts
    df.drop(columns=["Composite Weather Variable - Normal", "Composite Weather Variable - Cold", "Composite Weather Variable - Warm"], inplace=True)

    if(remove_outliers):
        for col in label_cols:    
            # remove outliers where any of the prices was 0
            print(df.shape)
            df = df[df[col] != 0]
            print(df.shape)
            df = df[df[f"Next Day {col}"] != 0]
            print(df.shape)
            #... and where the next day price is more than least 50% away from the current day's price
            df = df[abs(df[col] - df[f"Next Day {col}"])/df[col] < 0.5]
            print(df.shape)
    return df    

df = clean(df)
df.to_csv(Path("../data/processed/preprocessed_and_cleaned.csv"), index=False)
df.head()

(1802, 78)
(1802, 78)
(1802, 78)
(1789, 78)
(1789, 78)
(1789, 78)
(1789, 78)
(1784, 78)
(1784, 78)
(1783, 78)
(1783, 78)
(1762, 78)


Data Item,Gas Day,Aggregate LNG Importations - Daily Flow,Beach Including Norway - Daily Flow,Beach and IOG - Beach Delivery,Beach and IOG - Daily Flow,Composite Weather Variable - Actual,Demand - Cold,"Demand - Cold, (excluding interconnector and storage)",Demand - Warm,"Demand - Warm, (excluding interconnector and storage)",...,SMPSell D30 roll std,Day of Week,Is Weekday,Next Day Is Weekday,Day of Year,sin_DoY,cos_DoY,Next Day SAP,Next Day SMPBuy,Next Day SMPSell
5,2020-05-06,50.23730,142.20118,192.43848,192.43848,12.0645,245.399580,216.150490,145.432835,116.183744,...,0.005142,2,1,1,127,0.816538,-0.577292,0.4834,0.5187,0.4481
6,2020-05-07,53.59770,141.87295,195.47065,195.47065,13.4655,243.927941,214.341578,145.153439,115.567076,...,0.011180,3,1,1,128,0.806480,-0.591261,0.4756,0.5109,0.4403
7,2020-05-08,51.08822,135.19872,186.28694,186.28694,15.5400,171.000000,131.000000,154.000000,114.000000,...,0.010249,4,1,0,129,0.796183,-0.605056,0.4722,0.5075,0.4369
8,2020-05-09,53.28634,127.04213,180.32847,180.32847,15.0800,172.000000,134.000000,141.000000,104.000000,...,0.009697,5,0,0,130,0.785650,-0.618671,0.4615,0.4968,0.4262
9,2020-05-10,53.14522,127.29315,180.43837,180.43837,12.6200,264.115666,226.223933,180.336933,142.445199,...,0.009489,6,0,1,131,0.774884,-0.632103,0.4569,0.4922,0.4216


### Split the data into training and test sets
Various ways were investigated:
- Use the data before a cutoff date to train, and after that date to test. Designed to test whether the model will generalise to the most recent period, despite having been trained on earlier periods
- Split the data 50/50 by date. Use all of the earlier half, and a random half of the later half, to train. Use the remaining half of the later half to test. Designed as a compromise between the above approach and a random split
- Split the data randomly regardless of date

In [87]:

def split_train_test_on_date(df, split_date, discard_before_date):
    """
    Splits the DataFrame into training set (gas days before split date) and test set (ga days fron the split date on)

    :param df: The DataFrame to split.
    :param split_date: The date to split the DataFrame on.
    :param discard_before_date: Discard anything before this date. Added to exclude time of Covid lockdowns.
    :return: Tuple of (training set, testing set).
    """

    # Split the DataFrame into training and testing sets
    train_df = df[df['Gas Day'].between(discard_before_date, split_date, inclusive = "neither")]
    test_df = df[df['Gas Day'] >= split_date]

    return train_df, test_df

def split_with_test_half_of_last_half(df, discard_before_date):
    """
    Use all the earlier half, and half the later half, to train
    Use the other half of the later half to test
    """
    df2 = df[df['Gas Day'] >= discard_before_date]
    mid_date = df2['Gas Day'].mean()
    first_half = df2[df2['Gas Day'] < mid_date]
    second_half = df2[df2['Gas Day'] >= mid_date]
    
    train_df, test_df = train_test_split(second_half, test_size=0.5, shuffle=True)
    train_df = pd.concat([first_half, train_df])
    return train_df, test_df

def n_train_n_test(df, n_train, n_test, discard_before_date):
    """Split based on number or fraction of rows"""
    
    df = df[df['Gas Day'] >= discard_before_date]
    # Split the DataFrame into training and testing sets
    train_df, test_df = train_test_split(df, test_size=n_test, train_size=n_train, shuffle=True)
    
    return train_df, test_df

def get_X(df):
    ys = ["Next Day " + col for col in label_cols]
    df2 = df.drop(columns=ys)
    df2.drop(columns=["Gas Day"], inplace=True)
    
    return df2

train, test = split_with_test_half_of_last_half(df, '2023-09-01')
X_train = get_X(train)
X_test = get_X(test)


### Use Root Mean Squared Error as the measure of accuracy

This is appropriate to price forecasting because it penalises larger inaccuracies

In [88]:
# Root mean squared error - penalises larger errors more than smaller ones
def get_rmse(actuals, predictions):
    rmse =  np.sqrt(np.mean((predictions - actuals)**2))
    return round(rmse, 4)

def print_model_stats(model, X):

    # 1. Coefficients and intercept
    if hasattr(model, "coef_"):
        #print("Coefficients:", model.coef_)      # array of shape (n_features,)
        cdf = pd.DataFrame(model.coef_, X.columns, columns=['Coefficients'])
        cdf = cdf.sort_values(by='Coefficients', ascending=False)
        print(cdf)
    if hasattr(model, "intercept_"):
        print("Intercept:", model.intercept_)    # scalar (or array if multi-output)

    # 2. Model parameters
    print("Parameters:", model.get_params())

    # 3. Linear algebra internals
    if hasattr(model, "rank_"):
        print("Rank of design matrix:", model.rank_)
    if hasattr(model, "singular_"):
        print("Singular values of X:", model.singular_)

### Set up a framework to train models, and compare their performance on the test dataset against a naive predictor

The naive predictor takes the current day's System Average Price and System Marginal Prices as the predictions for the next day

In [89]:
TEST_ON_RANDOM = "Random"
TEST_ON_LATEST = "Latest "
TEST_ON_HALF_LATEST = "Half latest"

class Context:
    """Context for a model evaluation"""

    def __init__(self, model_type, test_set):
        self.model_type = model_type
        self.test_set = test_set

    def __repr__(self):
        return f"Context(model_type={self.model_type}, test_set={self.test_set})"
    
class Result:
    """Result of a model evaluation"""
    
    def __init__(self, context:Context, price_label, model_rmse, naive_rmse):
        self.context = context
        self.price_label = price_label
        self.model_rmse = model_rmse
        self.naive_rmse = naive_rmse
        self.timestamp = datetime.datetime.now()

    def __repr__(self):
        return f"GasPredictResult(context={self.context}, price_label={self.price_label}, model_rmse={self.model_rmse}, naive_rmse={self.naive_rmse}, timestamp={self.timestamp})"    

def get_y(df, col):
    return df["Next Day " + col]

def test_model(model, X, y):
    y_pred = model.predict(X)
    rmse = get_rmse(y, y_pred)
    return rmse

def train_and_test_model(model, df_train, df_test, col):
    X_train = get_X(df_train)
    X_test = get_X(df_test)
    y_train = get_y(df_train, col)
    y_test = get_y(df_test, col)
    #scaler = StandardScaler()
    #X_train_scaled = scaler.fit_transform(X_train)
    #X_test_scaled = scaler.fit_transform(X_test)
    X_train_scaled = X_train
    X_test_scaled = X_test

    model.fit(X_train_scaled, y_train)

    rmse_train = test_model(model, X_train_scaled, y_train)
    rmse_test = test_model(model, X_test_scaled, y_test)

    return model, rmse_train, rmse_test

def train_test_and_report_for_prices(model_factory, df_train: pd.DataFrame, df_test: pd.DataFrame, context:Context, print_model_stats=True):
    results = []
    for col in label_cols:
        # Instantiate model.
        model = model_factory()

        # Train and test it
        model, rmse_train, rmse_test = train_and_test_model(model, df_train, df_test, col)

        # Print model details
        if print_model_stats:
            X_train = get_X(df_train)
            print_model_stats(model, X_train)

        # Get naive prediction stats for comparison
        rmse_naive_train, rmse_naive_test = naive_predictions(df_train, df_test, col)

        print_results(col + " train", rmse_naive_train, rmse_train)
        print_results(col + " test", rmse_naive_test, rmse_test)

        testResult = Result(context, col, rmse_test, rmse_naive_test)
        results.append(testResult)
    return results

def naive_predictions(df_train, df_test, col):
    naive_predictions_train = df_train[col]
    actuals_train = df_train[f"Next Day {col}"]
    #mape_naive_train = get_mape(actuals_train, naive_predictions_train)
    #print(f"MAPE train (naive predictor) for {col}: {mape_naive_train}")
    rmse_naive_train = get_rmse(actuals_train, naive_predictions_train)
    #print(f"RMSE train (naive predictor) for {col}: {rmse_naive_train}")

    naive_predictions_test = df_test[col]
    actuals_test = df_test[f"Next Day {col}"]
    #mape_naive_test = get_mape(actuals_test, naive_predictions_test)
    #print(f"MAPE test (naive predictor) for {col}: {mape_naive_test}")
    rmse_naive_test = get_rmse(actuals_test, naive_predictions_test)
    #print(f"RMSE test (naive predictor) for {col}: {rmse_naive_test}")
    return rmse_naive_train, rmse_naive_test

def print_results(case, rmse_naive, rmse_model):
    headline = "Worse" if rmse_naive <= rmse_model else "Better"
    print(f"{case} - {headline} - model {rmse_model} v naive {rmse_naive}")


all_results = []

### Try linear regression models

...to predict each of SAP (System Average Price), SMPBuy (System Marginal Price - Buy) and SMPSell (System Marginal Price - Sell). This generally performs worse than the naive predictor in testing, especially using a date-based split

In [90]:
print ("Linear regression model:")
model_factory = lambda: LinearRegression()

print("Using random train-test split...")
context = Context("Linear regression", TEST_ON_RANDOM)
train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print("Using pure date-based train-test split...")
context = Context("Linear regression", TEST_ON_LATEST)
train, test = split_train_test_on_date(df, '2024-04-01', '2020-10-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print("Using half of the last half for testing...")
context = Context("Linear regression", TEST_ON_HALF_LATEST)
train, test = split_with_test_half_of_last_half(df, '2021-04-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print(all_results)

Linear regression model:
Using random train-test split...
SAP train - Better - model 0.387 v naive 0.457
SAP test - Better - model 0.3738 v naive 0.4551
SMPBuy train - Better - model 0.4567 v naive 0.5157
SMPBuy test - Better - model 0.486 v naive 0.5302
SMPSell train - Better - model 0.5181 v naive 0.5653
SMPSell test - Worse - model 0.6889 v naive 0.683
Using pure date-based train-test split...
SAP train - Better - model 0.4075 v naive 0.4965
SAP test - Worse - model 0.6024 v naive 0.0806
SMPBuy train - Better - model 0.4878 v naive 0.5645
SMPBuy test - Worse - model 0.8553 v naive 0.0966
SMPSell train - Better - model 0.5906 v naive 0.6496
SMPSell test - Worse - model 1.0567 v naive 0.0891
Using half of the last half for testing...
SAP train - Better - model 0.4338 v naive 0.5295
SAP test - Worse - model 0.1668 v naive 0.1217
SMPBuy train - Better - model 0.5217 v naive 0.602
SMPBuy test - Worse - model 0.1937 v naive 0.1433
SMPSell train - Better - model 0.6304 v naive 0.694
SMPSel

### Try a random forest model
Linear regression generally performed worse than the naive predictor in testing, especially using a date-based split, so let's try a random forest model. The hyperparameters for the best version were obtained by random search in the second code block below

In [91]:
from sklearn.ensemble import RandomForestRegressor
print ("Random forest model:")
#RandomForestRegressor(n_estimators = 500, min_samples_split = 2, min_samples_leaf= 2, max_features = 0.9, max_depth = 20, ccp_alpha = 0.0) # best from random searh
#RandomForestRegressor(n_estimators = 200, min_samples_split = 2, min_samples_leaf= 2, max_features = 0.7, max_depth = 20, ccp_alpha = 0.0) # best for SAP: SAP test - Better - model 0.77 v naive 0.8
model_factory = lambda: RandomForestRegressor(n_estimators = 500, min_samples_split = 2, min_samples_leaf= 2, max_features = 0.9, max_depth = 20, ccp_alpha = 0.0)

print("Using random train-test split...")
context = Context("Random forest", TEST_ON_RANDOM)
train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print("Using pure date-based train-test split...")
context = Context("Random forest", TEST_ON_LATEST)
train, test = split_train_test_on_date(df, '2024-04-01', '2020-10-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print("Using half of the last half for testing...")
context = Context("Random forest", TEST_ON_HALF_LATEST)
train, test = split_with_test_half_of_last_half(df, '2021-04-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

Random forest model:
Using random train-test split...
SAP train - Better - model 0.1873 v naive 0.4684
SAP test - Worse - model 0.4277 v naive 0.4188
SMPBuy train - Better - model 0.2198 v naive 0.5398
SMPBuy test - Worse - model 0.5015 v naive 0.4528
SMPSell train - Better - model 0.2788 v naive 0.6294
SMPSell test - Worse - model 0.5217 v naive 0.4868
Using pure date-based train-test split...
SAP train - Better - model 0.1972 v naive 0.4965
SAP test - Worse - model 0.1037 v naive 0.0806
SMPBuy train - Better - model 0.2322 v naive 0.5645
SMPBuy test - Worse - model 0.1225 v naive 0.0966
SMPSell train - Better - model 0.2874 v naive 0.6496
SMPSell test - Worse - model 0.1197 v naive 0.0891
Using half of the last half for testing...
SAP train - Better - model 0.2074 v naive 0.5299
SAP test - Worse - model 0.1219 v naive 0.1167
SMPBuy train - Better - model 0.2474 v naive 0.6021
SMPBuy test - Worse - model 0.1438 v naive 0.1428
SMPSell train - Better - model 0.3106 v naive 0.6939
SMPSel

### Next, try gradient boosting
Again the random forest improves in test slightly on a random split but not when trained on earlier data and tested on later. Let's try tree-based gradient boosting

In [92]:
from xgboost import XGBRegressor

print("Gradient Boosting model (XGBoost XGBRegressor):")

model_factory = lambda: XGBRegressor(
        n_estimators=200,
        max_depth=6,

        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.7,
        reg_alpha=0.0,
        reg_lambda=1.0,
        random_state=42
    )

print("Using random train-test split...")
context = Context("Gradient boosting", TEST_ON_RANDOM)
train, test = n_train_n_test(df, n_train=0.75, n_test=0.25, discard_before_date='2021-03-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print("Using pure date-based train-test split...")
context = Context("Gradient boosting", TEST_ON_LATEST)
train, test = split_train_test_on_date(df, '2024-04-01', '2020-10-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)

print("Using half of the last half for testing...")
context = Context("Gradient boosting", TEST_ON_HALF_LATEST)
train, test = split_with_test_half_of_last_half(df, '2021-04-01')
all_results += train_test_and_report_for_prices(model_factory, train, test, context, print_model_stats=False)


Gradient Boosting model (XGBoost XGBRegressor):
Using random train-test split...
SAP train - Better - model 0.0233 v naive 0.465
SAP test - Better - model 0.4091 v naive 0.4299
SMPBuy train - Better - model 0.028 v naive 0.5117
SMPBuy test - Better - model 0.5307 v naive 0.5418
SMPSell train - Better - model 0.0286 v naive 0.6091
SMPSell test - Worse - model 0.5973 v naive 0.5589
Using pure date-based train-test split...
SAP train - Better - model 0.0332 v naive 0.4965
SAP test - Worse - model 0.116 v naive 0.0806
SMPBuy train - Better - model 0.0373 v naive 0.5645
SMPBuy test - Worse - model 0.1506 v naive 0.0966
SMPSell train - Better - model 0.0401 v naive 0.6496
SMPSell test - Worse - model 0.1448 v naive 0.0891
Using half of the last half for testing...
SAP train - Better - model 0.0287 v naive 0.5299
SAP test - Worse - model 0.1216 v naive 0.1166
SMPBuy train - Better - model 0.032 v naive 0.6023
SMPBuy test - Worse - model 0.1469 v naive 0.1394
SMPSell train - Better - model 0.0

### Try Recurrent Neural Network

The gradient booster likewise did not perform any better than the naive predictor, especially when trained on the earlier data and tested on the later data. Let's try a neural net. For a simple time series, a Temporal Convolutional Net would be the obvious choice, but in this case we have a lot of market fundamentals to use as additional features so a RNN seems the better fit.

(1) Because of how the inputs need to be shaped into sequences, we'll load the data again, skipping the manually-engineered lag features and next-day labels. We'll still fill in missing actual Composite Weather Variables with the normal forecast, but won't delete the few with outlying prices in case the RNN is sophisticated enough to make good use of them.

In [93]:
#Reload with minimal preprocessing and cleaning
df = load_data()
df = preprocess(df, add_lags=False, add_labels=False)
df = clean(df, remove_outliers=False)
df = df.sort_values('Gas Day').reset_index(drop=True) # Should already be sorted, but just in case
df.head()

Processed 10 of 60 raw files
Processed 20 of 60 raw files
Processed 30 of 60 raw files
Processed 40 of 60 raw files
Processed 50 of 60 raw files
Processed 60 of 60 raw files


Data Item,Gas Day,Aggregate LNG Importations - Daily Flow,Beach Including Norway - Daily Flow,Beach and IOG - Beach Delivery,Beach and IOG - Daily Flow,Composite Weather Variable - Actual,Demand - Cold,"Demand - Cold, (excluding interconnector and storage)",Demand - Warm,"Demand - Warm, (excluding interconnector and storage)",...,"Storage, Short Range, Maximum potential flow","Storage, Short Range, Stock Levels","System Entry Flows, National, Forecast","System Entry Flows, National, Physical",Day of Week,Is Weekday,Next Day Is Weekday,Day of Year,sin_DoY,cos_DoY
0,2020-05-01,66.15110,135.26344,201.41454,201.41454,10.5824,268.090483,240.620483,160.371026,132.901026,...,0.0,0.0,247.492971,253.383732,4,1,0,122,0.863142,-0.504961
1,2020-05-02,58.78630,131.66283,190.44913,190.44913,11.3089,244.938804,217.103350,145.468324,117.632869,...,0.0,0.0,208.688512,229.860588,5,0,0,123,0.854322,-0.519744
2,2020-05-03,56.55015,141.57363,198.12378,198.12378,11.6531,242.854588,214.663679,144.375923,116.185014,...,0.0,0.0,221.676044,252.336835,6,0,1,124,0.845249,-0.534373
3,2020-05-04,52.82721,152.87412,205.70133,205.70133,11.9252,242.098717,213.561444,144.746098,116.208825,...,0.0,0.0,206.768342,231.939225,0,1,1,125,0.835925,-0.548843
4,2020-05-05,62.21188,134.50813,196.72001,196.72001,11.4803,247.112422,218.209695,145.956044,117.053317,...,0.0,0.0,201.921238,189.255071,1,1,1,126,0.826354,-0.563151


(2) Make the sequences, covering 30 days of the salient features

In [94]:

WINDOW_SIZE = 30

feature_cols = ['Composite Weather Variable - Actual', 'Demand Actual, NTS, D+1', 'Demand Forecast, NTS, hourly update', 'Interconnector - Daily Flow', 'Medium Storage - Actual Stock',
              'Medium Storage - Stock Level at Max Flow', 'Predicted Closing Linepack (PCLP1)', 
              'SAP', 'SMPBuy',	'SMPSell', 
              'Storage - Daily Flow','Storage - Delivery', 'Storage, Medium Range, Stock Levels', 'System Entry Flows, National, Forecast', 'System Entry Flows, National, Physical',
              'Day of Week','Is Weekday','Next Day Is Weekday','Day of Year']

def make_sequences(df, feature_cols):
    X, Y = [], []
    for i in range(len(df) - WINDOW_SIZE):
        X.append(df[feature_cols].iloc[i : i + WINDOW_SIZE].values)
        Y.append(df[label_cols].iloc[i + WINDOW_SIZE].values) # using SAP, SMPBuy and SMPSell as labels as before
    return np.array(X), np.array(Y)

X, y = make_sequences(df, feature_cols)

(3) Split sequentially into training, validate and test sets, so that the new gas days introduced at each stage are later than the days already seen. Then scale the sets individually.

In [95]:
train_size = int(0.7 * len(X))
val_size   = int(0.1 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val,   y_val   = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test,  y_test  = X[train_size+val_size:], y[train_size+val_size:]
print(f"Training records: {X_train.shape[0]}")
print(f"Validation records: {X_val.shape[0]}")
print(f"Test records: {X_test.shape[0]}")


n_feats = X_train.shape[2]
scaler = StandardScaler()
X_train_2d = X_train.reshape(-1, n_feats)
scaler.fit(X_train_2d)

def scale_split(X):
    X_2d = X.reshape(-1, n_feats)
    Xs = scaler.transform(X_2d)
    return Xs.reshape(-1, WINDOW_SIZE, n_feats)

#Take a copy of the unscaled test data for comparison against the naive predictor
X_test_unscaled = X_test.copy()

X_train = scale_split(X_train)
X_val   = scale_split(X_val)
X_test  = scale_split(X_test)


Training records: 1244
Validation records: 177
Test records: 357


(4) Train and test the model - unfortunately it doesn't fit into the framework for sklearn-type models

In [96]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

def make_rnn():
    #model = models.Sequential([
        #layers.LSTM(128, return_sequences=True, input_shape=(WINDOW_SIZE, n_feats)),
        #layers.Dropout(0.02),
        #layers.LSTM(64),
        #layers.Dropout(0.02),
        #layers.Dense(32, activation='relu'),
        #layers.Dense(3, name='multi_output')   # predicts [SAP, SMPBuy, SMPSell]
    #])
    model = models.Sequential([
        # Single, small LSTM — no return_sequences, so it only outputs the last hidden state
        layers.LSTM(32, input_shape=(WINDOW_SIZE, n_feats)),

        # (Optional) small dense “bottleneck” to pick up any non-linear mix
        layers.Dense(16, activation='relu'),

        # Multi-output head predicts [SAP, SMPBuy, SMPSell]
        layers.Dense(3, name='multi_output')
    ])


    model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
    )

    model.summary()
    return model

def train_and_test_rnn(model, context:Context):

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5
        )
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        callbacks=callbacks
    )

    # 5. Evaluate model RMSE on test set
    eval_results = model.evaluate(X_test, y_test, return_dict=True)
    model_rmse = eval_results['rmse']
    print(f"Overall RMSE: {model_rmse:.4f}")

    results = []
    #Individual RMSE for each target
    y_pred = model.predict(X_test)
    for i, name in enumerate(label_cols):
        #get model RMSE for each target
        rmse = get_rmse(y_test[:,i], y_pred[:,i])
        print(f"{name} RMSE: {rmse:.4f}")

        # get naive predictor RMSE based on the unscaled inputs
        feat_idx = feature_cols.index(name)
        y_pred_naive = X_test_unscaled[:, -1, feat_idx]
        y_true = y_test[:, i]
        naive_rmse = get_rmse(y_true, y_pred_naive)
        print(f"{name} naive predictor RMSE: {naive_rmse:.4f}")

        # add stats
        testResult = Result(context, name, rmse, naive_rmse)

        # add to the running list of results
        results.append(testResult)

    return results

model = make_rnn()
context = Context("RNN", TEST_ON_LATEST)
all_results += train_and_test_rnn(model, context)

c:\Users\mike\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 32)             │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_output (Dense)            │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,235 (28.26 KB)

 Trainable params: 7,235 (28.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - loss: 25.6282 - rmse: 5.0574 - val_loss: 3.6550 - val_rmse: 1.9118 - learning_rate: 0.0010
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 11.6664 - rmse: 3.4005 - val_loss: 2.4421 - val_rmse: 1.5627 - learning_rate: 0.0010
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.2651 - rmse: 2.0574 - val_loss: 1.3342 - val_rmse: 1.1551 - learning_rate: 0.0010
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.2940 - rmse: 1.5113 - val_loss: 0.8676 - val_rmse: 0.9314 - learning_rate: 0.0010
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 1.4136 - rmse: 1.1876 - val_loss: 0.8562 - val_rmse: 0.9253 - learning_rate: 0.0010
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.0811 - rmse: 1.0380 - val_loss: 0.6339 - val_rmse: 0.7962 - learning_rate: 0.0010
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 1.1201 - rmse: 1.0536 - val_loss: 0.5899 - val_rmse: 0.7680 - learning_rat

### Finally, try reframing RNN with a residual model

The RNN is still testing worse than naive predictor, indicating that the network is not learning anything from the additional fields that adds anything to the current day prices.
As a final option, try a residual model that predicts the delta from the current day's price



In [97]:

def make_rnn_residual():
    # 1) Inputs
    inputs = layers.Input(shape=(WINDOW_SIZE, n_feats))

    # 2) Core LSTM
    x = layers.LSTM(32)(inputs)
    x = layers.Dense(16, activation='relu')(x)

    # 3) Delta prediction head (predict tomorrow’s Δ for each series)
    delta = layers.Dense(3, name='delta')(x)  
    #   outputs [ΔSAP, ΔSMPBuy, ΔSMPSell]
    idxs_of_labels = [feature_cols.index(c) for c in label_cols]
    # 4) Grab today's values from the last timestep of the sequence
    #    This gives shape (batch, 3) corresponding to [SAP_t, SMPBuy_t, SMPSell_t].
    last_vals = layers.Lambda(lambda z: tf.gather(z[:, -1, :], idxs_of_labels, axis=1),
                            name='last_vals')(inputs)

    # 5) Add skip-connection: tomorrow = today + predicted Δ
    outputs = layers.Add(name='residual_output')([last_vals, delta])

    # 6) Assemble and compile
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
    )

    model.summary()
    return model

model = make_rnn_residual()
context = Context("Residual RNN", TEST_ON_LATEST)
all_results += train_and_test_rnn(model, context)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 30, 19)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 32)        │      6,656 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 16)        │        528 │ lstm_7[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last_vals (Lambda)  │ (None, 3)         │          0 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ delta (Dense)       │ (None, 3)         │         51 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual_output     │ (None, 3)         │          0 │ last_vals[0][0],  │
│ (Add)               │                   │            │ delta[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,235 (28.26 KB)

 Trainable params: 7,235 (28.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 22.6008 - rmse: 4.7458 - val_loss: 7.9864 - val_rmse: 2.8260 - learning_rate: 0.0010
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 10.6863 - rmse: 3.2601 - val_loss: 2.0446 - val_rmse: 1.4299 - learning_rate: 0.0010
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3.0966 - rmse: 1.7544 - val_loss: 1.1042 - val_rmse: 1.0508 - learning_rate: 0.0010
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.0733 - rmse: 1.0344 - val_loss: 0.8299 - val_rmse: 0.9110 - learning_rate: 0.0010
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.0092 - rmse: 0.9961 - val_loss: 1.0283 - val_rmse: 1.0141 - learning_rate: 0.0010
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.7420 - rmse: 0.8596 - val_loss: 1.1450 - val_rmse: 1.0700 - learning_rate: 0.0010
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.6416 - rmse: 0.7998 - val_loss: 1.1750 - val_rmse: 1.0840 - learning_rat

### Pick the best performing model type
No model outperformed the naive predictor so I'll have to choose the least bad one to tune based on the gathered results


In [98]:
# sort by the delta (model_rmse - naive_rmse)
#Filter results to only include "Latest" data test set
filtered_results = [r for r in all_results if r.context.test_set == TEST_ON_LATEST]

sorted_results = sorted(filtered_results, key=lambda r: r.naive_rmse - r.model_rmse, reverse=True)

for r in sorted_results:
    difference = r.naive_rmse - r.model_rmse
    print(f"{r.context.model_type} ({r.price_label}): difference over naive RMSE = {difference:.4f}")

Random forest (SAP): difference over naive RMSE = -0.0231
Random forest (SMPBuy): difference over naive RMSE = -0.0259
Random forest (SMPSell): difference over naive RMSE = -0.0306
Gradient boosting (SAP): difference over naive RMSE = -0.0354
Gradient boosting (SMPBuy): difference over naive RMSE = -0.0540
Gradient boosting (SMPSell): difference over naive RMSE = -0.0557
RNN (SAP): difference over naive RMSE = -0.2876
RNN (SMPBuy): difference over naive RMSE = -0.2993
RNN (SMPSell): difference over naive RMSE = -0.3708
Linear regression (SAP): difference over naive RMSE = -0.5218
Residual RNN (SMPBuy): difference over naive RMSE = -0.5924
Residual RNN (SAP): difference over naive RMSE = -0.6225
Linear regression (SMPBuy): difference over naive RMSE = -0.7587
Residual RNN (SMPSell): difference over naive RMSE = -0.7765
Linear regression (SMPSell): difference over naive RMSE = -0.9676


Random forest came out best

In [ ]:
#Random search for hyperparameter tuning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

param_grid = {
    'n_estimators':     [100, 200, 500],
    'max_depth':        [None, 10, 20],
    'min_samples_split':[2],
    'min_samples_leaf': [2],
    'max_features':     [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 1.0],
    'ccp_alpha':        [0.0, 0.001]
}
#param_grid = {
#    'n_estimators':     [100, 200],
#    'max_depth':        [None, 10, 20],
#    'min_samples_split':[2, 5],
#    'min_samples_leaf': [1, 2],
#    'max_features':     ['sqrt'],
#    'ccp_alpha':        [0.001, 0.01]
#}
X_train = get_X(train)
for col in label_cols:

    rf = RandomForestRegressor(
        random_state=42,
        n_jobs=-1,
        
        oob_score=True   # optional: get out‑of‑bag score on your train set
    )
    #grid = GridSearchCV(
    #    estimator=rf,
    #    param_grid=param_grid,
    #    cv=5,                             # 5‑fold CV on X_train/y_train
    #    scoring='neg_root_mean_squared_error',
    #    n_jobs=-1,
    #    verbose=3
    #)
    rand_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_grid,             
        n_iter=50,                                  
        cv=5,
        scoring='neg_root_mean_squared_error',      
        n_jobs=-1,
        verbose=3,
        random_state=42
    )

    y_train = get_y(train, col)
    rand_search.fit(X_train, y_train)

    print("Best hyperparameters:", rand_search.best_params_)
    print("Best CV RMSE on train set: {:.4f}".format(-rand_search.best_score_))


    X_test = get_X(test)
    y_test = get_y(test, col)
    # -----------------------------------------------------------------------------
    # 5. Evaluate the best model on the TEST set
    # -----------------------------------------------------------------------------
    best_model = rand_search.best_estimator_

    rmse_test = test_model(best_model, X_test, y_test)
    rmse_naive_train, rmse_naive_test = naive_predictions(train, test, col)

    print_results(col + " test", rmse_naive_test, rmse_test)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.7, 'max_depth': None, 'ccp_alpha': 0.001}
Best CV RMSE on train set: 0.4899
SAP test - Worse - model 0.1277 v naive 0.1166
Fitting 5 folds for each of 50 candidates, totalling 250 fits


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 1.0, 'max_depth': 20, 'ccp_alpha': 0.0}
Best CV RMSE on train set: 0.7879
SAP test - Worse - model 0.78 v naive 0.78
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.7, 'max_depth': None, 'ccp_alpha': 0.001}
Best CV RMSE on train set: 0.8540
SMPBuy test - Worse - model 0.86 v naive 0.83
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.5, 'max_depth': None, 'ccp_alpha': 0.001}
Best CV RMSE on train set: 0.8906
SMPSell test - Worse - model 0.9 v naive 0.83